In [1]:
from google.colab import files
uploaded=files.upload()

Saving fake news data.csv to fake news data.csv


In [ ]:
import pandas as pd
import numpy as np
df = pd.read_csv('/content/fake news data.csv')
df=df.drop(['Unnamed: 4','Unnamed: 5'],axis='columns')
df

In [ ]:
df.head()
df.shape
df.info()
df.describe()
df.isnull().sum()
df.columns

In [ ]:
df = df.copy()
df['Body'] = df['Body'].fillna('')
df.isnull().sum()

In [ ]:
df['News'] = df['Headline']+df['Body']
df
df.head()
df.columns

In [ ]:
df = df.drop(['URLs','Headline','Body'],axis =1)
df
df.columns

Index(['Label'], dtype='object')

In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import re
ps = PorterStemmer()
def wordopt(text):
    text = re.sub('[^a-zA-Z]', ' ',text)
    text = text.lower()
    text = text.split()
    text = [ps.stem(word) for word in text if not word in stopwords.words('english')]
    text = ' '.join(text)
    return text
df['News'] = df['News'].apply(wordopt)
df.head()

In [ ]:
X = df['News']
Y = df['Label']
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.25)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorization = TfidfVectorizer()
xv_train = vectorization.fit_transform(x_train)
xv_test = vectorization.transform(x_test)

In [ ]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model.fit(xv_train,y_train)
y_pred = model.predict(xv_test)
from sklearn.metrics import accuracy_score
score = accuracy_score(y_test,y_pred)
print('Accuracy of LR model is ', score)

In [ ]:
from sklearn.svm import SVC
model = SVC(kernel='linear')
model.fit(xv_train,y_train)
y_pred = model.predict(xv_test)
from sklearn.metrics import accuracy_score
score = accuracy_score(y_test,y_pred)
print('Accuracy of SVM model is ', score)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(random_state=0)
model.fit(xv_train, y_train)
y_pred = model.predict(xv_test)
from sklearn.metrics import accuracy_score
score = accuracy_score(y_test,y_pred)
print('Accuracy of RFC model is ', score)

In [ ]:
def fake_news_det(news):
    input_data = {"text":[news]}
    new_def_test = pd.DataFrame(input_data)
    new_def_test["text"] = new_def_test["text"].apply(wordopt)
    new_x_test = new_def_test["text"]
    #print(new_x_test)
    vectorized_input_data = vectorization.transform(new_x_test)
    prediction = model.predict(vectorized_input_data)
    if prediction == 1:
        print("Not a Fake News")
    else:
        print("Fake News")

In [ ]:
fake_news_det('Four ways Bob Corker skewered Donald Trump')
fake_news_det('JetNation FanDuel League; Week 4 of readers think this story is Fact. Add your two cents.(Before Its News)Our FanDuel league is back again this week. Here are the details:$900 in total prize money. $250 to the winner. $10 to enter.Remember this is a one week league, pick your lineup against the salary cap and next week if you want to play again you can pick a completely different lineup if you want.Click this link to enter — http://fanduel.com/JetNation You can discuss this with other NY Jets fans on the Jet Nation message board. Or visit of on Facebook.Source: http://www.jetnation.com/2017/09/27/jetnation-fanduel-league-week-4/')

Not a Fake News
Fake News
